Profiling D3M+Auctus augmentation
===
In this notebook I will briefly analyze the the augmentation candidates provided
by Auctus when queried using the respective D3M table. I will be using the small
D3M set of seed tables rather than the full one.

In [ ]:
%cd ~/work/study-data-lakes/

In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import polars as pl
import json
from pathlib import Path
from src.auctus.utils import profile_dataset
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Setting working folder
working_folder = Path(".")
root_data_folder = Path(working_folder, "data")
data_folder = Path(root_data_folder, "soda-data-lake/a-d3m/a-d3m_small")
assert data_folder.exists()

## Profiling the data lake

In [5]:
dataset_list = list(data_folder.iterdir())

df_info_overall = pd.DataFrame()

for dataset_path in tqdm(dataset_list, total=len(dataset_list)):
    df_info = profile_dataset(dataset_path=dataset_path, engine="polars")
    df_info_overall = pd.concat([df_info_overall, df_info])

df_info_overall.to_csv("info_a-d3m_small.csv", index=False)


100%|██████████| 8/8 [00:43<00:00,  5.42s/it]


## Reading the csv with the statistics

In [6]:
df_stats = pd.read_csv("info_a-d3m_small.csv")

I'm taking a look at the content of the file. 

In [8]:
df_stats.head()

,ds_name,candidate_name,left_on,right_on,merged_rows,scale_factor,left_unique_keys,right_unique_keys,left_rows,left_cols
0,299_libras_move_MIN_METADATA,datamart.socrata.data-edmonton-ca.mnwt-r49h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,299_libras_move_MIN_METADATA,datamart.url.e8ad58ccedbd437b96f4a36de3f4f8a7,"['ycoord1', 'xcoord1']","['Lat', 'Long']",360.0,1.0,328.0,285.0,360.0,92.0
2,299_libras_move_MIN_METADATA,datamart.url.e8ad58ccedbd437b96f4a36de3f4f8a7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,299_libras_move_MIN_METADATA,datamart.socrata.data-pr-gov.qzxu-awbt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,299_libras_move_MIN_METADATA,datamart.zenodo.4464734.5aa81844-52b9-4202-b77...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A lot of the rows contain `NaN` values: these are cases that have failed for 
various reasons (candidate table not found, mismatch between the join columns and
more). 
These cases should be dropped from the study.

In [9]:
df_stats_clean = df_stats.dropna(how="any")
print(f"Keeping {len(df_stats_clean)/len(df_stats)*100:.2f} % of the dataset.")
display(df_stats_clean.head())

Keeping 76.38 % of the dataset.


Another important column to look at is `scale_factor`: this column is obtained
by dividing the value in `merged_rows` by `left_rows`. This shows how much larger
the merged table would be if a left join is executed using the given left and 
right keys. 

Here, no aggregation function is applied to 1-to-many matches, which means that 
each match in the left column is replicated for each match in the right column. 

The scale factor is necessarily >= 1, since this is a left join. A scale factor
of exactly 1 means that no matches were found between the left and right table
(an inner join in this case would be an empty set). 

In [14]:
print(f"{len(df_stats.loc[df_stats['scale_factor']==1])} cases result in empty joins.")

valid_joins = df_stats.loc[df_stats["scale_factor"]>1]

print(f"{len(valid_joins)} cases have `scale_factor` > 1")

84 cases result in empty joins.
23 cases have `scale_factor` > 1


Out of the 23 cases with a valid scale factor, how many of them lead to an 
explosion in the number of rows in the merged table? If I choose a threshold of 
20 (i.e. each row in the left table is replicated on average 20 times), this is
the result.

Note: 20 is probably already a lot. 

In [20]:
valid_over_20 = valid_joins.loc[valid_joins["scale_factor"] > 20][
    ["ds_name", "candidate_name", "merged_rows", "scale_factor"]
    ]
print(f"{len(valid_over_20)} datasets have `scale_factor` > 20")
display(valid_over_20)

10 datasets have `scale_factor` > 20


,ds_name,candidate_name,merged_rows,scale_factor
56,LL0_200_pbc,datamart.zenodo.6974483.3d9901cd-8a55-48ee-b2f...,161442.0,386.224880
57,LL0_200_pbc,datamart.socrata.data-usaid-gov.mm3i-pcnd,93585.0,223.887560
58,LL0_200_pbc,datamart.socrata.data-usaid-gov.ibtf-2a34,70992.0,169.837321
60,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.www-data-act-gov-au.c5h9-9bh7,614041.0,20.293509
63,LL1_736_population_spawn_MIN_METADATA,datamart.zenodo.4243547.ece93f17-c7b0-48a2-b05...,929658.0,30.724370
69,LL1_736_population_spawn_MIN_METADATA,datamart.zenodo.6621762.6876a8dd-25e0-4c4b-918...,1879351.0,62.110880
80,LL1_gasoline_incl_aviation_production_industry,datamart.socrata.opendata-utah-gov.nbv6-7v56,82006.0,569.486111
107,political_instability_MIN_METADATA,datamart.socrata.data-usaid-gov.v799-4hug,153041.0,26.541970
108,political_instability_MIN_METADATA,datamart.socrata.data-usaid-gov.75qt-x4hk,258046.0,44.753035
109,political_instability_MIN_METADATA,datamart.socrata.data-usaid-gov.r3z6-2jv8,2437076.0,422.663198


Some of these datasets have a scale factor in the hundreds: these joins are
completely useless, seeing how all rows in the first table would be replicated
hundreds of times. No useful information can be obtained in such cases. 

What about the datasets that do not have this problem? 

In [21]:
valid_sub_20 = valid_joins.loc[valid_joins["scale_factor"] <= 20][
    ["ds_name", "candidate_name", "merged_rows", "scale_factor"]
    ]
print(f"{len(valid_sub_20)} datasets have `scale_factor` <= 20")
display(valid_sub_20)

13 datasets have `scale_factor` <= 20


,ds_name,candidate_name,merged_rows,scale_factor
59,LL0_200_pbc,datamart.socrata.data-usaid-gov.4gye-9wpi,3960.0,9.473684
62,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.data-ct-gov.ybkg-w5x2,30344.0,1.002842
64,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.www-data-act-gov-au.65z9-5mfa,102992.0,3.403794
66,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.health-data-ny-gov.gnvg-m62r,30517.0,1.008560
70,LL1_736_population_spawn_MIN_METADATA,datamart.zenodo.5142676.979356df-53c5-4842-824...,422025.0,13.947551
71,LL1_736_population_spawn_MIN_METADATA,datamart.zenodo.5021480.47623c54-e1a8-4716-b80...,34893.0,1.153183
72,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.data-ct-gov.4z97-pa4q,32597.0,1.077302
77,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.data-ny-gov.6amx-2pbv,368959.0,12.193767
79,LL1_736_population_spawn_MIN_METADATA,datamart.socrata.opendata-utah-gov.7iu3-rv9s,30741.0,1.015963
81,LL1_gasoline_incl_aviation_production_industry,datamart.socrata.opendata-maryland-gov.x7h2-rnih,198.0,1.375000


There seem to be at least a few datasets that were joined and have a promising
scale factor, and might warrant further investigation.

## Conclusion

This notebook was intended to illustrate the issues that are bound to occur if 
the augmentation candidates provided by Auctus are naively joined on the seed 
table. Even ignoring the missing datasets, the majority of the augmentation 
candidates would lead to empty join sets, while out of the remaining cases, many 
of the joins result in an explosion of the size of the left table. 

In short, the candidates provided by Auctus are not to be used directly and should
first be vetted to remove cases that would lead to issues downstream. 